In [1]:
import pandas as pd
import numpy as np
import glob

pd.options.mode.chained_assignment = None  # default='warn'

# Preprocess pipeline for single file


In [132]:
df = pd.read_csv('../Data/72243012960/2000_72243012960.csv')

/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,42,49,50,56,57,58,59,60,61,64,65,66,67,72,74,75,76) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [133]:
df

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,MW2,MW3,MW4,MW5,OC1,OE1,OE2,OE3,REM,EQD
0,72243012960,2000-01-01T00:00:00,3,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",SY-MT,IAH,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN076AAXX 01004 72243 32966 00000 10144 2007...,Q01 00002PRCP06N01 00000JPWTH 0
1,72243012960,2000-01-01T00:53:00,C,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",FM-15,IAH,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET067KIAH 010053Z 00000KT 10SM CLR 12/07 A300...,N01 00000JPWTH 0
2,72243012960,2000-01-01T01:00:00,9,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",NSRDB,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72243012960,2000-01-01T01:53:00,C,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",FM-15,IAH,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET067KIAH 010153Z 00000KT 10SM CLR 10/07 A300...,N01 00000JPWTH 0
4,72243012960,2000-01-01T02:00:00,9,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",NSRDB,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19235,72243012960,2000-12-31T21:53:00,C,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",FM-15,IAH,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET080KIAH 312153Z 08010KT 10SM -RA OVC050 06/...,Q01+000003PRSWM1Q02 003PRSWM1N01 00020JPWTH 0
19236,72243012960,2000-12-31T22:00:00,9,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",NSRDB,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19237,72243012960,2000-12-31T22:53:00,C,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",FM-15,IAH,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET088KIAH 312253Z 07010KT 9SM -RA OVC046 04/0...,Q01+000003PRSWM1Q02 003PRSWM1N01 00020JPWTH 0
19238,72243012960,2000-12-31T23:00:00,9,29.98,-95.36,29.0,"HOUSTON INTERCONTINENTAL AIRPORT, TX US",NSRDB,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
df[~df['AT1'].isnull()]['AT1']

71       MW,01,FG  ,5
177      MW,16,RA  ,1
322      MW,16,RA  ,5
373      MW,16,RA  ,5
435      MW,16,RA  ,1
             ...     
18894    MW,16,RA  ,5
18943    MW,16,RA  ,5
18996    MW,16,RA  ,5
19050    MW,16,RA  ,5
19155    MW,01,FG  ,5
Name: AT1, Length: 234, dtype: object

In [122]:
def ParseDF(df):
    # Select columns we care
    
    # PARSE
    # Split DATE
    datetime = pd.to_datetime(df['DATE'])
    df['DATE_year'] = datetime.dt.year
    df['DATE_month'] = datetime.dt.month
    df['DATE_day'] = datetime.dt.day
    df['DATE_hour'] = datetime.dt.hour
    df['DATE_year_month_day'] = df[['DATE_year', 'DATE_month', 'DATE_day']].apply(tuple, axis=1)
    
    columns_parse = ['WND','CIG','VIS','TMP','DEW']
    # Split Wind WND data
    df[['WND_angle','WND_angle_quality','WND_wind_observation','WND_speed','WND_speed_quality']] = df.WND.str.split(',', expand=True)
    # Split CIG data
    df[['CIG_ceiling_height', 'CIG_ceiling_quality','CIG_determ', 'CIG_CAVOK']] = df.CIG.str.split(',', expand=True)
    # Split VIS data
    df[['VIS_distance','VIS_distance_quality', 'VIS_observation','VIS_observation_quality']] = df.VIS.str.split(',', expand=True)
    # Split TMP data
    df[['TMP_temp_celsius','TMP_temp_quality']] = df.TMP.str.split(',', expand=True)
    # SPlit DEW data
    df[['DEW_temp_celsius','DEW_quality']] = df.DEW.str.split(',', expand=True)
    # Split SLP data
    df[['SLP_hectopascal','SLP_quality']] = df.SLP.str.split(',', expand=True)
    
    def getAT(input):
        #print(type(input))
        # individual data look like: "AU,13,BR ,5", or "none"
        if type(input) == str:
            try:
                return int(input.split(',')[1])
            except:
                return 0
        else:
            return 0
    
    def thunder_check(array, check=3):
        if check in array:
            return 1
        else:
            return 0
        
    def tornado_check(array, check=10):
        if check in array:
            return 1
        else:
            return 0
        
    def rain_check(array, check=16):
        if check in array:
            return 1
        else:
            return 0
        
    def checkBadWeather(row):
        return 1 if np.sum(row)>0 else 0
    
    # AT
    df['AT1'] = df['AT1'].apply(getAT)
    df['AT2'] = df['AT2'].apply(getAT)
    df['AT3'] = df['AT3'].apply(getAT)
    #df['AT4'] = df['AT4'].apply(getAT)
    df['AT'] = list(df[['AT1','AT2','AT3']].to_numpy())
    df['AT_binary'] = df['AT'].apply(checkBadWeather)
    df['AT_thunder'] = df['AT'].copy()
    df['AT_thunder'] = df['AT_thunder'].apply(thunder_check)
    df['AT_tornado'] = df['AT'].copy()
    df['AT_tornado'] = df['AT_tornado'].apply(tornado_check)
    df['AT_rain'] = df['AT'].copy()
    df['AT_rain'] = df['AT_rain'].apply(rain_check)

    # Drop parsed raw data
    df.drop(columns_parse, axis=1, inplace=True)
    return df

def KeepRelevant(df):
    columns = ['STATION', 'DATE','DATE_year','DATE_month','DATE_day','DATE_hour','DATE_year_month_day', 'LATITUDE',
               'LONGITUDE', 'ELEVATION', 'WND_angle', 'WND_speed', 'CIG_ceiling_height', 'VIS_distance',
               'TMP_temp_celsius', 'DEW_temp_celsius','SLP_hectopascal','AT','AT_rain','AT_thunder','AT_tornado','AT_binary']
    df = df[columns]
    return df

def ReplaceMissingNumericWithNeighbor(df, column, to_replace=[999,9999,99999,999999], value=np.nan, rescale=1):
    '''Replace Missing values with NaN, Fill with method='ffill'''
    df[column] = df[column].astype(int)
    df[column].replace(to_replace=to_replace, value=np.nan, inplace=True)
    # Fill NaN
    df[column] = df[column].fillna(method='ffill')
    df[column] = df[column].fillna(method='bfill')
    
    if rescale != 1:
        df[column] = df[column]/rescale
        
    return df

# FIll thunder for all day
def Fill_thunder_day(df):    
    month_day_s = list(df[df['AT_thunder']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_thunder']=1
    return df

def Fill_rain_day(df):    
    month_day_s = list(df[df['AT_rain']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_rain']=1
    return df

# FIll thunder for all day
def Fill_tornado_day(df):    
    month_day_s = list(df[df['AT_tornado']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_tornado']=1
    return df

def Fill_AT_binary_day(df):
    month_day_s = list(df[df['AT_binary']==1]['DATE_year_month_day'])
    df.loc[df['DATE_year_month_day'].isin(month_day_s), 'AT_binary']=1
    return df


def DataframePreprocessingPipeline(df):
    # Parse
    df = ParseDF(df)
    df = KeepRelevant(df)
    
    # Replace & Rescale
    df = ReplaceMissingNumericWithNeighbor(df, 'WND_angle')
    df = ReplaceMissingNumericWithNeighbor(df, 'WND_speed', rescale=10)
    df = ReplaceMissingNumericWithNeighbor(df, 'CIG_ceiling_height')
    df = ReplaceMissingNumericWithNeighbor(df, 'VIS_distance')
    df = ReplaceMissingNumericWithNeighbor(df, 'TMP_temp_celsius',rescale=10)
    df = ReplaceMissingNumericWithNeighbor(df, 'DEW_temp_celsius', rescale=10)
    df = ReplaceMissingNumericWithNeighbor(df, 'SLP_hectopascal', rescale=10)
    
    df = Fill_thunder_day(df)
    df = Fill_tornado_day(df)
    df = Fill_AT_binary_day(df)
    df =  Fill_rain_day(df)
    
    return df

In [113]:
# # Read StationReference.csv
# StationReference = pd.read_csv('../Data/StationReference.csv')
# print(StationReference.shape)

In [114]:
# # US
# #query = StationReference[StationReference['cc'] == 'US']
# # California
# # query = StationReference[StationReference['admin1'] == 'California']
# # San Fran
# query = StationReference[StationReference['name'] == 'San Francisco']


# StationReference_toget = list(query['filename'])
# Station_sf_filenames = [s + '.csv' for s in StationReference_toget]
# Station_sf_filenames

# Go through files in data folder

In [115]:
# year = str(2020)

# inputpathcollection = '../Data/'+year+'/*.csv'
# inputpath = '../Data/'+year+'/'
# outputpath = '../DataProcessed/'+year+'/'
# for file in glob.glob(inputpathcollection):
#     fname = file[13:]
#     #print(fname_nopath)
#     #print(fname)
#     if str(fname) in Station_sf_filenames:
#         df_raw = pd.read_csv(inputpath+fname)
#         #print('get sf')
#         # Preprocess pipeline
#         try:
#             df = DataframePreprocessingPipeline(df_raw)
#             outputpath_full = outputpath + fname
#             df.to_csv(outputpath_full, index=False)
#         except:
#             continue

In [126]:
year = str(72243012960)#Houston
#year = str(72494023234)#SFO

inputpathcollection = '../Data/'+year+'/*.csv'
inputpath = '../Data/'+year+'/'
outputpath = '../DataProcessed/'+year+'/'

df_all = pd.DataFrame()

for file in glob.glob(inputpathcollection):
    #print(file)
    fname = file[13:]
    #print(fname_nopath)
    print(fname)
    df_raw = pd.read_csv(file)
    df = DataframePreprocessingPipeline(df_raw)
    df_all = pd.concat([df_all,df])

012960/2011_72243012960.csv
012960/2008_72243012960.csv


/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (42,43,47,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


012960/2007_72243012960.csv


/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (43,47,51,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


012960/2019_72243012960.csv


...

/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (41,45,74) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


012960/2002_72243012960.csv


/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (41,53,71,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


012960/2014_72243012960.csv


/opt/anaconda3/envs/aviation/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (43,52,53,58,85,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [129]:
df_all = df_all.sort_values('DATE')

In [130]:
df_all

,STATION,DATE,DATE_year,DATE_month,DATE_day,DATE_hour,DATE_year_month_day,LATITUDE,LONGITUDE,ELEVATION,...,CIG_ceiling_height,VIS_distance,TMP_temp_celsius,DEW_temp_celsius,SLP_hectopascal,AT,AT_rain,AT_thunder,AT_tornado,AT_binary
0,72243012960,2000-01-01T00:00:00,2000,1,1,0,"(2000, 1, 1)",29.98000,-95.36000,29.0,...,22000.0,16000.0,14.4,7.2,1016.4,"[0, 0, 0]",0,0,0,0
1,72243012960,2000-01-01T00:53:00,2000,1,1,0,"(2000, 1, 1)",29.98000,-95.36000,29.0,...,22000.0,16000.0,12.2,7.2,1017.0,"[0, 0, 0]",0,0,0,0
2,72243012960,2000-01-01T01:00:00,2000,1,1,1,"(2000, 1, 1)",29.98000,-95.36000,29.0,...,22000.0,16000.0,12.2,7.2,1017.0,"[0, 0, 0]",0,0,0,0
3,72243012960,2000-01-01T01:53:00,2000,1,1,1,"(2000, 1, 1)",29.98000,-95.36000,29.0,...,22000.0,16000.0,10.0,6.7,1017.4,"[0, 0, 0]",0,0,0,0
4,72243012960,2000-01-01T02:00:00,2000,1,1,2,"(2000, 1, 1)",29.98000,-95.36000,29.0,...,22000.0,16000.0,10.0,6.7,1017.4,"[0, 0, 0]",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,72243012960,2022-04-30T05:53:00,2022,4,30,5,"(2022, 4, 30)",29.98438,-95.36072,27.5,...,518.0,16093.0,23.3,20.0,1010.2,"[0, 0, 0]",0,0,0,0
3878,72243012960,2022-04-30T05:59:00,2022,4,30,5,"(2022, 4, 30)",29.98438,-95.36072,27.5,...,518.0,16093.0,23.3,20.0,1010.2,"[0, 0, 0]",0,0,0,0
3879,72243012960,2022-04-30T06:53:00,2022,4,30,6,"(2022, 4, 30)",29.98438,-95.36072,27.5,...,488.0,16093.0,23.3,20.0,1009.6,"[0, 0, 0]",0,0,0,0
3880,72243012960,2022-04-30T07:32:00,2022,4,30,7,"(2022, 4, 30)",29.98438,-95.36072,27.5,...,22000.0,16093.0,23.3,20.0,1009.6,"[0, 0, 0]",0,0,0,0


In [131]:
df_all.to_csv('../DataProcessed/houston_airport_cleaned_generalized_2000-2022.csv',index=False)

In [21]:
df_exp = df_all.copy()
df_exp.drop(['STATION','DATE','AT','DATE_year','DATE_year_month_day','ELEVATION'], axis=1,inplace=True)
y = df_exp['AT_binary']
X = df_exp.drop('AT_binary',axis=1)

In [ ]:
import sklearn

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# model = DecisionTreeClassifier(random_state=0).fit(X_train,y_train)
# accuracy_score(y_test, model.predict(X_test))

In [ ]:
param_max_depth = np.arange(2,5) #6
param_min_samples_leaf = np.arange(1,2) #5

# Initiate array
Training_Scores = []
Test_Scores = []
best_model_score = {'train':0, 'test':0}
best_param = {}

for depth in param_max_depth:
    train_array = []
    test_array=[]
    for leaf in param_min_samples_leaf:
        # Fit this model
        DTC = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=leaf)
        DTC.fit(X, y)
        # Get this model scores
        training_score = np.mean(cross_val_score(DTC, X,y, cv=5))
        test_score = accuracy_score(DTC.predict(X_test), y_test)
        train_array.append(training_score)
        test_array.append(test_score)
        
        # Store best model
        if training_score > best_model_score['train']:
            best_model_score['train'] = training_score
            best_model_score['test'] = test_score
            best_param['min_samples_leaf'] = leaf
            best_param['max_depth'] = depth
    
    Training_Scores.append(train_array)
    Test_Scores.append(test_array)

Training_Scores # ([1,2,3],[1,2,3],[1,2,3]). Row:depth, Col:leaf

In [ ]:
DTC = DecisionTreeClassifier(max_depth=3, min_samples_leaf=1)
DTC.fit(X_train, y_train)

In [ ]:
print("Majority:", sum(y_test)/len(y_test))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(DTC.predict(X_test), y_test)

In [ ]:
df_all

In [ ]:
1- sum(df_all['AT_binary']) / len(df_all)

In [ ]:
df_raw